In [38]:
import torch 
from torchvision import transforms 
from torchvision import datasets 
from torch.utils.data import DataLoader 
import torch.nn.functional as F 
import torch.optim as optim
import time

## 1、Prepare Dataset

In [39]:
batch_size = 64

## Convert the PIL Image to Tensor.
transform = transforms.Compose([
    transforms.ToTensor(), # 图像转变为ToTensor，取值变为0-1
    transforms.Normalize((0.1307, ), (0.3081, )) 
    # 0.1307是均值mean, 0.3081是标准差std
    # 这两个数是根据 MNIST数据集的特征得到的经验值
])

## 构造loader
train_dataset = datasets.MNIST(root='./dataset/mnist/', 
                               train=True, 
                               download=True, 
                               transform=transform) # transform放入到了数据集中，这样每次取出来的数据都会直接做变换
train_loader = DataLoader(train_dataset, 
                          shuffle=True, 
                          batch_size=batch_size)
test_dataset = datasets.MNIST(root='./dataset/mnist/' , 
                              train=False, 
                              download=True, 
                              transform=transform)
test_loader = DataLoader(test_dataset, 
                         shuffle=False, 
                         batch_size=batch_size)

![eGnr5U](https://gitee.com/pxqp9W/testmarkdown/raw/master/imgs/2020/07/eGnr5U.png)

```python
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307, ), (0.3081, )) 
])
```

将28×28的0-255的像素值，转换为1×28×28在[0，1]之间的张量，其中1表示 **通道** 数目
![AzuJQM](https://gitee.com/pxqp9W/testmarkdown/raw/master/imgs/2020/07/AzuJQM.png)
数据归一化，其中的均值和标准差是根据MNIST数据集的特征确定的

![mXTFiV](https://gitee.com/pxqp9W/testmarkdown/raw/master/imgs/2020/07/mXTFiV.png)

## 2、Design Model 

In [40]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.l1 = torch.nn.Linear(784, 512) 
        self.l2 = torch.nn.Linear(512, 256) 
        self.l3 = torch.nn.Linear(256, 128) 
        self.l4 = torch.nn.Linear(128, 64) 
        self.l5 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784) # view函数可以改变tensor的形状，变为784列, 如果是torch.view(-1)则原张量会变成一维的结构。
        x = F.relu(self.l1(x)) # 784->512
        x = F.relu(self.l2(x)) # 512->256
        x = F.relu(self.l3(x)) # 256->128
        x = F.relu(self.l4(x)) # 128->64
        return self.l5(x) # 64->10，这里要注意最后一层没有用激活函数，因为后面调用torch.nn.CrossEntropyLoss()的时候自带激活函数
model = Net()

## 3、Construct Loss and Optimizer

In [41]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) # momentum表示动量

- SGD方法的一个缺点是，其更新方向完全依赖于当前的batch，因而其更新十分不稳定。解决这一问题的一个简单的做法便是引入momentum。
- 这里是用了优化了的SGD算法：SGD with Momentum
- Momentum在梯度下降的过程中加入了惯性，使得梯度方向不变的维度上速度变快，梯度方向有所改变的维度上的更新速度变慢，这样就可以加快收敛并减小震荡
- momentum即动量，**它模拟的是物体运动时的惯性，即更新的时候在一定程度上保留之前更新的方向，同时利用当前batch的梯度微调最终的更新方向。这样一来，可以在一定程度上增加稳定性，从而学习地更快，并且还有一定摆脱局部最优的能力**

## 4、Train & Test

In [42]:
## 将训练过程封装到函数
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data 
        optimizer.zero_grad()
        
        # forward + backward + update
        outputs = model(inputs) 
        loss = criterion(outputs, target) 
        loss.backward() 
        optimizer.step()
        running_loss += loss.item() 
        
        if batch_idx % 300 == 299: 
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300)) 
            running_loss = 0.0

In [43]:
## 将测试过程也封装为函数
def test():
    correct = 0 
    total = 0
    with torch.no_grad(): # Test过程中不需要计算梯度，可以使用“with torch.no_grad()”
        for data in test_loader:
            images, labels = data 
            outputs = model(images) 
            _, predicted = torch.max(outputs.data, dim=1) # torch.max用来找行中的最大值所在下标 dim1表示沿着第1个维度找（行是第0个维度，列是第1个维度）
            total += labels.size(0) # labels是一个N*1的向量，取size()得到的是元组(N,1),size(0)返回的是N
            correct += (predicted == labels).sum().item() # 预测正确的数量，predicted == labels进行的是张量之间的比较运算
    print('Accuracy on test set: %d %%' % (100 * correct / total))

> `with torch.no_grad():`中的运算代码不会计算梯度

## Main Function

In [44]:
if __name__ == '__main__':
    start = time.time()
    for epoch in range(10):
        train(epoch) 
        test()
    end = time.time()
    print("Total time: ", end - start , "s")

[1,   300] loss: 2.263
[1,   600] loss: 1.187
[1,   900] loss: 0.442
Accuracy on test set: 90 %
[2,   300] loss: 0.317
[2,   600] loss: 0.278
[2,   900] loss: 0.228
Accuracy on test set: 93 %
[3,   300] loss: 0.192
[3,   600] loss: 0.163
[3,   900] loss: 0.159
Accuracy on test set: 95 %
[4,   300] loss: 0.125
[4,   600] loss: 0.128
[4,   900] loss: 0.116
Accuracy on test set: 96 %
[5,   300] loss: 0.103
[5,   600] loss: 0.099
[5,   900] loss: 0.087
Accuracy on test set: 96 %
[6,   300] loss: 0.080
[6,   600] loss: 0.078
[6,   900] loss: 0.069
Accuracy on test set: 97 %
[7,   300] loss: 0.057
[7,   600] loss: 0.062
[7,   900] loss: 0.064
Accuracy on test set: 97 %
[8,   300] loss: 0.047
[8,   600] loss: 0.050
[8,   900] loss: 0.051
Accuracy on test set: 97 %
[9,   300] loss: 0.040
[9,   600] loss: 0.039
[9,   900] loss: 0.042
Accuracy on test set: 97 %
[10,   300] loss: 0.030
[10,   600] loss: 0.033
[10,   900] loss: 0.036
Accuracy on test set: 97 %
Total time:  97.4272677898407 s
